reasons for different flux values:
- input parameters (obviously)
- float point rounding
- interpolation schemes
- grid resolution
- threading or cpu behavior

all can cascade into diferent values

if we did fit separate slopes/offsets, we would be implicitly saying:

> “the electron thermodynamics change with positron fraction in a way not captured by the model.”

that's a different paper.

1. how many different types of turbulence heating models is there

2. investigate optically thick ipole. why does ipole not do well with optically thick?

3. investigate if grmonty struggles with high munit/hotter and is that why angelo did not see any problems with sgr a*?

expect idential I, Q, U, V to 5+ sigfigs

bitwise identical results

fluxes within 5-10%

similar polarization structure

consistent image morphology


timesteps can be generated from this formula: i.e. timestep 25000 = 5000 (in filename):

```py
import h5py, pathlib as p

for f in sorted(p.Path("/work/vmo703/ipole_outputs/Ma-0.5_5000")
                .glob("MAD_spin-0.5_t5000_RBETA_pos0.h5")):
    with h5py.File(f, "r") as H:
        tM = float(H["/fluid_header/t"][()]) # time in code units ~ M
        F  = float(H["/Ftot_unpol"][()])
    print(f.name, "  t[M] =", tM, "  Ftot[Jy] =", F)
```

with output:

```
MAD_spin-0.5_t5000_RBETA_pos0.h5   t[M] = 25000.006108902577   Ftot[Jy] = 0.5576462116678979
```

In [ ]:
import h5py, pathlib as p

for f in sorted(p.Path("/work/vmo703/ipole_outputs/Ma-0.5_5000")
                .glob("MAD_spin-0.5_t5000_RBETA_pos0.h5")):
    with h5py.File(f, "r") as H:
        tM = float(H["/fluid_header/t"][()]) # time in code units ~ M
        F  = float(H["/Ftot_unpol"][()])
    print(f.name, "  t[M] =", tM, "  Ftot[Jy] =", F)


MAD_spin-0.5_t5000_RBETA_pos0.h5   t[M] = 25000.006108902577   Ftot[Jy] = 0.5576462116678979


SANE+0.94_4000 (timestep: 20000) only has one value for MunitOffset and MunitSlope:

```py
offset_slope_key = {
    "20000": {
        "MunitOffset": {
            "MAD+0.94": [
                4.5e+24, 2.5e+24, 3e+24, 2.2e+24
            ],
            "MAD-0.5": [
                1.3e+25, 8e+24, 1e+25, 8.5e+24
            ],
            "SANE-0.5": [
                5e+28,1.9e+28, 2.9e+28, 1.7e+28
            ],
            "SANE+0.94": [
                1.5e+27
            ]
        },
        "MunitSlope": {
            "MAD+0.94": [
                1.2, 1.2, 1.2, 1.1
            ],
            "MAD-0.5": [
                2.8, 2.9, 2.5, 2.1
            ],
            "SANE-0.5": [
                28.0, 30.0, 22.0, 21.0
            ],
            "SANE+0.94": [
                0.72
            ]
        }
    }
}
```

5.2.1 footnote 3:

Each individual pixel in the image plane (normal to the observer) has an
$I_{\nu} = dF_{\nu} / d\Omega$ in cgs units, and $F_{tot} \approx 2.4 \times 10^{−23} erg\, s^{−1}\, cm^{−2}\, Hz^{−1} = 2.4\, Jy$ is just the sum of all the $dF_{\nu}$’s. Thus, multiply each cgs intensity colored pixel value by $57.9$ to get its flux density in Jy.

new PROBLEMS:

- using MunitUsed from IPOLE and using that for GRMONTY gives 0 flux - SANE only

conversion from $\nu*L_{\nu} \rightarrow$ maybe find solid angle?

5.2.1 footnote 3 (Comparison of Sgr A* with GRMHD simulations) for conversion between $I_{\nu}$ and $F_{\nu}$

set the munit in GRMONTY to match a data point near $230 GHz$

sane discoveries:

- `/params/L_unit` $\approx 9.6 \times 10^{14} cm$
    - should have been luminosity unit ($erg\, s^{-1}$)
    - $9.6 \times 10^{14}$ just so happens to be _gravitational radius_ $r_g = GM/c^2$
- `/params/nu_unit` was missing outright
    - should have been $\approx 10^{14}-10^{15}\, Hz$
- `/output/lnu` ranged from $-12 \rightarrow 9.6$
    - meant to be $log_{10}(\nu / \nu_{unit})$, but $\nu_{unit}$ wasn't applied

- all photon energies were being stored as **dimensionless code units**. ergo, when i read those numbers literally, they were smaller by $\sim 10^{27}-10^{39}$

- GRMONTY outputs unscaled νLν in code units.
- The raw flux will never match 0.5 Jy without calibration.
- The scale factor K is the physical normalization factor.
- You need K every time unless you modify GRMONTY or bake Munit into every run.
- This has nothing to do with “bias” in the Compton sampler.

- do electron-positron pairs annihilate ?
    - yes, but the rate is negligible in RIAFs $\rightarrow$ no 511 keV line, no significant energy loss
- why do we care about pairs in IPOLE?
    - because they control **polarization**, not annihilation
- does GRMONTY track positrons ?
    - no, because compton scattering is charge-insensitive
- so why generate spectra of both in IPOLE?
    - to model **faraday rotation / conversion**, which depends on the difference $n_{e^-} - n_{e^+}$

- what does GRMHD actually compute??
    - mass, momentum, and energy conservation (field equations)
    - $E + v \times B = 0$ (maxxwell's equations in ideal MHD)
    - curved spacetime dynamics using Kerr metric

but GRMHD **DOES NOT** give you electron temperature!! it only evolves **ions** + **magnetic field** + total internal energy.

- GRMHD gives you ONE temperature, but electriosn and ions are NOT the same temperature!!
    - electrons cool efficiently
    - ions stay hot (simulations don't know this)

##### you must *impose* an electron temperature model after the fact $\rightarrow$ **electron heating prescriptions**

**electron heating is sub-grid physics**

- GRMHD doesn't resolve plasma microphysics like
    - turbulence cascade
    - magnetic reconnection
    - landau damping
    - collisionless shock heating

so we use empirical or theoretical prescriptions:

**RBETA ($R-\beta$ model)**

electron temperature scales inversly with the plasma beta:

$T_e \propto \beta^{-1}$

because low-beta $\rightarrow$ magnetic field dominate $\rightarrow$ reconnection heats electrons

**CRITBETA**

electron temperature changes behavior when $\beta$ crosses a threshold

**wJET / funnel modes**

- electrons in the jet funnel are heated differently because the plasma is
    - low density
    - highly magnetized
    - collisionless

this step converts GRMHD into a **physical electron distribution**, which you then ray-trace

without this step, **the emission is meaningless**

the plasma emits light through synchrotron emission and absorption!

1. synchrotron emission
    - electrons spiraling in magnetic fields produce broadband radiation

- the spectrum depends on
    - $n_e$
    - $B$
    - electron temperature $\theta_e$
    - viewing angle

2. synchrotron absoption
    - the same population absorbs photons

3. compton scattering
    - low energy photons are boosted by hot electrons $\rightarrow$ creates high energy tail

4. pair annihilation (negligible)
    - not important in RIAFs, but theoretically exists

for polarized radiative transfer in curved spacetime you must solve the **GR radiative transfer equation** along photon geodesics

$\frac{d \mathbf{S}}{d \lambda} = \mathbf{J} - \mathbf{KS}$

where $\mathbf{S}$ is the stokes vector: $(I,Q,U,V)$

- $\mathbf{K}$ contains
    - absorption
    - faraday rotation
    - faraday conversion

- synchrotron emissivity creates $I,Q,U,V$
- faraday rotation rotates $Q/U \rightarrow$ twists EVPA
- faraday conversion mixes $Q \longleftrightarrow V$
- pairs suppress faraday rotation

GRMONTY simulates photons by:
1. emit photon packets based on synchrotron emissivity
2. scatter them via compton scattering
3. weight each packet by "bias" to keep the monte carlo estimator unbiased

> monte carlo methods approximate the solution to the boltzmann equation

- GRMONTY gives you
    - energy spectra (nuLnu)
    - high energy compton tail
    - total flux at 230 GHz

but NOT polarization !!

in IPOLE you can solve the transfer equation **deterministically** , not with monte carlo

- IPOLE:
    - integrates geodesics backward from camera
    - evolves stokes parameters
    - accounts for GR effects (aberration, redshift, parallel transport)
    - supports distinct electron + positron populations
    - generates polarized images and spectra

1. GRMHD simulation
    - mass density, ion temperature, B-field, velocity field
2. electron heating prescription
    - electron temperature everywhere
    - plus positron fraction
3. radiative physics
    - synchrotron emissivity
    - absorption
    - faraday rotation/conversion
    - compton scattering
4. radiative transport solver
    - GRMONTY $\rightarrow$ SEDs
    - IPOLE $\rightarrow$ images + polarization
5. observational comparison
    - match 240 GHz flux
    - match EVPA, Q/I, U/I, V/I
    - compare to EHT constraints

expected SED behavior:

- the presence of a compton tail mainly depends on:
    - optical depth $\tau$
    - the electron temperature distribution $\theta_e$
    - where the hot electrons occur relative to where photons originate
    - the geometry (MAD vs SANE)

> the heating model *influences* $\tau$ and $\theta_e$, but doesn't guarantee a tail. this is why CRITBETA in SANE *did* exactly what it should: very weak/no tail.

---

1. SANE + RBETA
- what RBETA does
    + heats electrons proportional to (1/$\beta$)
    + so low-$\beta$ regions (more magnetized) get **hotter electrons**
    + the funnel wall gets *some* heating, but density is *still low*

- expected physics
    + moderate electron temps ($\theta_e \sim$ few-10)
    + $\tau$ still small in SANE $\rightarrow$ scattering weak

- expected SED
    + *weak to modest* compton tail
    + sometimes visible as a slight extension above the mm bump
    + never huge unless mdot is artificially high

RBETA in SANE is the "classic" SANE EHT model - it doesn't give strong tails unless you tune it aggresively.

---

2. SANE + CRITBETA
- what CRITBETA does
    + heats electrons **only** in $\beta < \beta_-$ crit zones (magnetically dominated)
    + in the disk midplane ($\beta \gg 1$), CRITBETA yields **much cooler electrons**
    + SANE disks are **high $\beta$ everywhere except the funnel edge**

- expected physics
    + $\theta_e$ cool in scattering regions
    + $\tau$ extremely small
    + hot electrons exist, but density there $\sim 0 \rightarrow$ irrelevant to radiative output

- expected SED
    + **NO compton tail or extremely weak tail**
    + pure mm bump shape
    + steep falloff toward NIR/X-ray

> CRITBETA gives stronger tails only in **MAD**, not SANE

---

3. SANE wJET (RBETA or CRITBETA)
- wJET boosts heating in **funnel / jet-sheath regions**, where SANE ordinarily has:
    + low density
    + low $\tau$
    + but *geometrically favorable* scattering paths

- expected physics
    + hotter sheath electrons ($\theta_e$ boosted)
    + still small $\tau \rightarrow$ number of scatterings small
    + but higher energy seed photons can get upscattered

- expected SED
    + **moderate compton tail**
    + stronger than RBETA or CRITBETA alone
    + but still not "big" unless you crank the jet heating too hard

> wJET exists *specifically* to fix the "SANE CRITBETA underheats electrons" problem

---

4. MAD + RBETA
- now geometry changes **everything**, MAD has:
    + higher density in funnel walls
    + higher $\tau$ in jet sheath
    + much stronger magnetic flux
    + more energy in low-$\beta$ regions

- expected physics
    + very hot electrons
    + moderate to high $\tau$
    + many scattering opportunities

- expected SED
    + **strong compton tail**
    + sometimes **too strong** for M87
    + can overshoot NIR/X-ray power
    + often disfavored in modern EHT comparisons
        - MAD+RBETA tends to overproduce high energy emisison

---

5. MAD + CRITBETA
- many moderns papers love this model because
    + MAD has many $\beta < \beta_-$ crit zones $\rightarrow$ CRITBETA heats a *lot* of the flow
    + higher density in funnel wall $\rightarrow$ strong scattering
    + higher $\tau$ along lines of sight
    + hotter electrons in the jet sheath region

- expected SED
    + **moderate but realistic compton tail**
    + matches M87 high energy emission much better
    + less extreme than RBETA
    + more physically motivated (electron heating in reconnection-driven regions)

> many groups prefer MAD+CRITBETA for M87-like systems

---

6. MAD + wJET (RBETA or CRITBETA)
- this contribution produces:
    + very hot sheath electrons
    + moderate-to-high $\tau$
    + large scattering depth
    + favorable geometry for multiple scatterings

- expected SED
    + **very strong compton tail**
    + sometimes too strong unless tuned
    + should be used with care: easy to overshoot NIR/X-ray power
    + but very useful if you WANT a pronounced high frequency component

> wJET in MAD is for "jet-bright" systems or exploring extreme models, not conservative fits

---

# FULL COMPARISON TABLE

| model               | expected tail strength       | why                                        |
| ------------------- | ---------------------------- | ------------------------------------------ |
| **SANE + CRITBETA** | ❌ none / extremely weak    | cool electrons + tiny $\tau$               |
| **SANE + RBETA**    | ⚠️ weak                     | slight heating in low-$\beta$ zones        |
| **SANE + WJET**     | ⭐ moderate                 | artificial heating in funnel               |
| **MAD + CRITBETA**  | ⭐⭐ moderate / realistic   | hot electrons + good $\tau$                |
| **MAD + RBETA**     | ⭐⭐⭐ strong               | overheats funnel; sometimes too much      |
| **MAD + WJET**      | ⭐⭐⭐⭐ very strong       | hot sheath + high $\tau$ + strong geometry |

---

# the most important conceptual takeaway

> **you should NOT expect a compton tail in every configuration**
>
> **CRITBETA in SANE is *supposed* to produce weak/no tail**
>
> **RBETA and WJET generate tails only if $\tau$ and $\theta_e$ are high enough**
>
> **MAD geometry amplifies ALL heating prescriptions**


| disk     | model        | tail?              | why                                                                 |
| -------- | ------------ | ------------------ | ------------------------------------------------------------------- |
| **SANE** | **CRITBETA** | usually none?      | electrons cooler; moderate $\sigma$; small scattering depth         |
| **SANE** | **RBETA**    | weak               | hotter electrons near inner region                                  |
| **SANE** | **wJET**     | strong             | jet electrons get very hot                                          |
| **MAD**  | **CRITBETA** | none               | CRITBETA suppresses electron heating in magnetized flow             |
| **MAD**  | **RBETA**    | (weak) maybe?      | still too magnetized; little scattering                             |
| **MAD**  | **wJET**     | potentially huge?  | funnel $\sigma$ is enormous; heated jet produces compton power-law  |


to systematically test new jet model, this is the correct progression:

1. MAD + CRITBETA (no jet)

confirm no tail

2. MAD + RBETA (no jet)

confirm maybe tiny tail or none

3. MAD + wJET (weak jet)

confirm spectrum $\approx$ CRITBETA (good)

4. MAD + wJET (stronger jet)

| parameter                                       | purpose                                                   |
| ----------------------------------------------- | --------------------------------------------------------- |
| `jet_sigma_cut`                                 | only apply jet heating when $\sigma > \sigma_{cut}$       |
| `jet_beta_cut`                                  | ignore regions with large gas pressure (disk)             |
| `jet_thetae`                                    | funnel electron temperature ceiling / override            |
| `jet_ne_mult`                                   | optional scaling of electron density in jets              |
| `sigma_transition`                              | smooth transition between disk model and jet model        |
| `constant_beta_e0`, `constant_beta_e0_exponent` | controls heating strength in magnetically dominated zones |


| paper                                                                                                                                                                          | what it says about MAD, RBETA / 2-temperature electron treatments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Two-temperature, Magnetically Arrested Disc simulations of the jet from the supermassive black hole in M87** — Chael, Narayan & Johnson (2018/19) ([arXiv][1])               | This is one of the first full 2-temperature GRRMHD simulations of a MAD disk + jet. The authors explicitly compare two different electron-heating prescriptions (turbulent cascade vs reconnection) and show that **in strongly magnetized (“jet/funnel”) regions**, electron temperatures become very high, which heavily influences the resulting emission spectrum. This demonstrates that applying a naive uniform heating law (or “simple” RBETA) across a MAD can lead to unphysical or at least strongly model-dependent results. ([arXiv][1]) |
| **Comparison of the ion-to-electron temperature ratio prescription: GRMHD simulations with electron thermodynamics** — Mizuno, Fromm, Porth, Younsi et al. (2021) ([arXiv][2]) | This paper studies the commonly used R–β prescription (i.e. RBETA-like) *in the context of MAD flows*. They compare R–β to more physically motivated electron-heating prescriptions, and find that while R–β can sometimes reproduce similar 230 GHz images, **the consistency is fragile** when jets/funnel regions are present. The authors point out that in MAD flows, simple R–β assumptions may not capture the complex physics of electron heating, especially in magnetically dominated zones. ([arXiv][2])                                   |
| **Survey of radiative, two‑temperature magnetically arrested simulations of the black hole M87* I: Turbulent electron heating** — Chael (2025) ([arXiv][3])                    | This is a more recent, comprehensive survey of 2-temperature, radiative MAD simulations including synchrotron, bremsstrahlung, and Compton, using sub-grid heating motivated by plasma turbulence / reconnection. The paper shows how **sensitive** model observables (SEDs, polarization, emission structure) are to the heating prescription — especially in the jet/funnel, and emphasizes the uncertainty in choosing a “correct” electron heating model for MAD states. ([arXiv][3])                                                             |

[1]: https://arxiv.org/abs/1810.01983 "Two-temperature, Magnetically Arrested Disc simulations of the jet from the supermassive black hole in M87"
[2]: https://arxiv.org/abs/2106.09272 "Comparison of the ion-to-electron temperature ratio prescription: GRMHD simulations with electron thermodynamics"
[3]: https://arxiv.org/abs/2501.12448 "Survey of Radiative, Two-Temperature Magnetically Arrested Simulations of the Black Hole M87* I: Turbulent Electron Heating"


models already tested (hayleys thesis and shaum's poster):

|      model       | mode | i[$^\circ$] | $a_*$ | $R_{high}$ | $R_{low}$ | $f$ | $\beta_c$ |
| ---------------- | ---- | ----------- | ----- | ---------- | --------- | --- | --------- |
| Critical-$\beta$ | MAD  | $163^\circ$ | -0.5  |  -         | -         | 0.5 | 1.0       |
| Critical-$\beta$ | MAD  | $163^\circ$ | +0.94 |  -         | -         | 0.5 | 1.0       |
| Critical-$\beta$ | MAD  | $163^\circ$ | -0.5  |  -         | -         | 0.5 | 0.1       |
| Critical-$\beta$ | MAD  | $163^\circ$ | +0.94 |  -         | -         | 0.5 | 0.1       |
| Critical-$\beta$ | SANE | $163^\circ$ | -0.5  |  -         | -         | 0.5 | 1.0       |
| Critical-$\beta$ | SANE | $163^\circ$ | +0.94 |  -         | -         | 0.5 | 1.0       |
| R-$\beta$        | MAD  | $163^\circ$ | -0.5  | 20         | 1         |  -  | 1.0       |
| R-$\beta$        | MAD  | $163^\circ$ | +0.94 | 20         | 1         |  -  | 1.0       |
| R-$\beta$        | SANE | $163^\circ$ | -0.5  | 20         | 1         |  -  | 1.0       |
| R-$\beta$        | SANE | $163^\circ$ | +0.94 | 20         | 1         |  -  | 1.0       |
| R-$\beta$        | MAD  | $163^\circ$ | -0.5  | 40         | 1         |  -  | 1.0       |
| R-$\beta$        | MAD  | $163^\circ$ | +0.94 | 40         | 1         |  -  | 1.0       |
| R-$\beta$        | MAD  | $163^\circ$ | -0.5  | 80         | 1         |  -  | 1.0       |
| R-$\beta$        | MAD  | $163^\circ$ | +0.94 | 80         | 1         |  -  | 1.0       |
| R-$\beta$        | MAD  | $163^\circ$ | -0.5  | 160        | 1         |  -  | 1.0       |
| R-$\beta$        | MAD  | $163^\circ$ | +0.94 | 160        | 1         |  -  | 1.0       |


**table 1b: final physical model grid for M87**

| heating model     | accretion state | $i\,[^\circ]$ | $a_*$ | $R_{\rm high}$ | $R_{\rm low}$ | $f$  | $\beta_c$ | status  |
|-------------------|-----------------|---------------|-------|----------------|---------------|------|-----------|---------|
| Critical-$\beta$  | MAD             | 163           | −0.5  | N/A            | N/A           | 0.5  | 0.01      | pending |
| Critical-$\beta$  | MAD             | 163           | +0.94 | N/A            | N/A           | 0.5  | 0.01      | pending |
| Critical-$\beta$  | SANE            | 163           | −0.5  | N/A            | N/A           | 0.5  | 0.1       | pending |
| Critical-$\beta$  | SANE            | 163           | +0.94 | N/A            | N/A           | 0.5  | 0.1       | pending |
| Critical-$\beta$  | SANE            | 163           | −0.5  | N/A            | N/A           | 0.5  | 0.01      | pending |
| Critical-$\beta$  | SANE            | 163           | +0.94 | N/A            | N/A           | 0.5  | 0.01      | pending |
| R-$\beta$         | MAD             | 163           | −0.5  | 40             | 1             | N/A  | 1.0       | pending |
| R-$\beta$         | MAD             | 163           | +0.94 | 40             | 1             | N/A  | 1.0       | pending |
| R-$\beta$         | SANE            | 163           | −0.5  | 40             | 1             | N/A  | 1.0       | pending |
| R-$\beta$         | SANE            | 163           | +0.94 | 40             | 1             | N/A  | 1.0       | pending |
| R-$\beta$         | MAD             | 163           | −0.5  | 160            | 1             | N/A  | 1.0       | pending |
| R-$\beta$         | MAD             | 163           | +0.94 | 160            | 1             | N/A  | 1.0       | pending |
| R-$\beta$         | SANE            | 163           | −0.5  | 160            | 1             | N/A  | 1.0       | pending |
| R-$\beta$         | SANE            | 163           | +0.94 | 160            | 1             | N/A  | 1.0       | pending |


**evaluation matrix (applied to all models in table 1b):**
- timesteps: $t = 20{,}000M,\;25{,}000M,\;30{,}000M$
- plasma composition: electron–ion ($e–i$), electron–positron ($e\pm$)
- jet heating: off, on
- radiative transfer:
  - $\texttt{IPOLE}$: polarimetric images $\left( I, Q, U, V \right)$, $P/I$, $V/I$, EVPA
  - $\texttt{GRMONTY}$: broadband spectra (subset of models)
- flux normalization: automated $\mathcal{M}$ tuning to $F_{230\,\mathrm{GHz}} = 0.5$ Jy


**table 2: critical-$\beta$ model parameters in the literature**

| target  | $f$ | $\beta_c$ | plasma | radiative code | key constraints used                  | reference                   |
|--------|-----|-----------|--------|----------------|----------------------------------------|-----------------------------|
| Sgr A* | 0.1 | 0.01      | e–i    | IPOLE          | 230 GHz flux, polarization             | [Anantua2020][1]            |
| Sgr A* | 0.1 | 0.1       | e–i    | IPOLE          | 230 GHz flux, polarization             | [Anantua2020][1]            |
| Sgr A* | 0.1 | 1.0       | e–i    | IPOLE          | 230 GHz flux, polarization             | [Anantua2020][1]            |
| Sgr A* | 0.5 | 0.01      | e–i    | IPOLE          | 230 GHz flux, polarization             | [Anantua2020][1]            |
| Sgr A* | 0.5 | 0.1       | e–i    | IPOLE          | 230 GHz flux, polarization             | [Anantua2020][1]            |
| Sgr A* | 0.5 | 1.0       | e–i    | IPOLE          | 230 GHz flux, polarization             | [Anantua2020][1]            |
| Sgr A* | 0.5 | 1.0       | e–i    | IPOLE          | EHT image + polarization constraints   | [EHT 2022 Sgr A paper V][2] |
| M87    | 0.5 | 1.0       | e–i    | IPOLE          | 230 GHz flux, polarization             | [Anantua2024][3]            |


[1]: https://arxiv.org/abs/2001.11556
[2]: https://arxiv.org/abs/2311.09478
[3]: https://ui.adsabs.harvard.edu/abs/2024AAS...24320612W/abstract

previously tested ($f,\; \beta_c$) values currently in the literature where most of the Sgr A* investigation comes from the initial introduction of the critical-$\beta$ model in [Anantua2020][1]
